In [1]:
#make-training-data.ipynb
#
#by Joe Hahn
#joe.hahn@oracle.com
#3 July 2018
#
#generate training data

In [16]:
N_train = 5#15000
x_half_width = 0.5
radius = 3.5
box_half_width = 7.1
jitter = 0.0
rn_seed = 13

In [7]:
import numpy as np
rn_state = np.random.RandomState(seed=rn_seed)
def make_xo_dict(x_half_width, radius, box_half_width, jitter, id=-1):
    #get dot's cartesian coordinates
    x = np.random.uniform(low=-box_half_width, high=box_half_width)
    y = np.random.uniform(low=-box_half_width, high=box_half_width)
    #get dot's class
    dot_class = 'B'
    r = np.sqrt(x**2 + y**2)
    if (r < radius):
        dot_class = 'O'
    if (np.abs(x) < x_half_width) or (np.abs(y) < x_half_width):
        dot_class = 'X'
    #rotate coordinate system by 45 degrees
    rot_angle = np.pi/4.0
    c = np.cos(rot_angle)
    s = np.sin(rot_angle)
    xr =  x*c + y*s
    yr = -x*s + y*c
    #add gaussian noise aka jitter to dots' positions
    xn = xr + np.random.normal(scale=jitter)
    yn = yr + np.random.normal(scale=jitter)
    #compute dot's polar coordinates
    r = np.sqrt(xn**2 + yn**2)
    angle = np.arctan2(yn, xn)
    xo = {'id':id, 'x':xn, 'y':yn, 'r':r, 'angle':angle, 'class':dot_class}
    return xo

In [15]:
xo = make_xo_dict(x_half_width, radius, box_half_width, jitter)
xo

{'angle': 1.1252169051134904,
 'class': 'O',
 'id': -1,
 'r': 3.204270266505195,
 'x': 1.3809789635229892,
 'y': 2.8914088336166235}

In [ ]:
np.random.normal?

In [ ]:
#number of dots in training dataset
N_train = 5#15000

#half-thickness of the x
x_half_width = 0.5

#radius of the O
radius = 3.5

#box half-width before 45 degree rotation
box_half_width = 7.1

#set jitter=scale of the gaussian noise, to make the class boundaries slightly fuzzy
jitter = 0.0

#set seed for random number generator
rn_seed = 13

#set debug=True to see debugging output
debug =  False

#generate the xo dataset
print 'generating xo data...'
initial_id = 0
from helper_fns import *
train = make_xo_data(N_train, initial_id, x_half_width, radius, box_half_width, jitter, rn_seed, debug)
print 'number of training records = ', len(train)

##save training data as csv file
#import pandas as pd
#pd.set_option('display.expand_frame_repr', False)
#print train.head(5)
#train.to_csv('data/train.txt', sep='|', index=False, header=False)

In [ ]:
train